In [8]:
from AHI_AC import *
res = 0.02

row_AHI = round(120/res)
col_AHI = round(120/res)

AHI_lat = np.linspace(60 - res/2 ,-60 + res/2,row_AHI)
AHI_lon = np.linspace(85 + res/2 ,205 - res/2,col_AHI)

target ='/data01/people/liwei/AC_Result/'

AHI_VZA,AHI_VAA = read_AHI_VA(res)
Landmask = read_Landmask(res)
AHI_AL = read_AHI_AL(res)
Water_idx = get_water_idx_2(Landmask)
interpolation = LUT_interpolation(5)
fn1_5, fn2_5, fn3_5 = interpolation.function_dict[interpolation.band]()
interpolation = LUT_interpolation(6)
fn1_6, fn2_6, fn3_6 = interpolation.function_dict[interpolation.band]()


In [10]:

def SR_Band5():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','sir','01',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band5)(fn1_5, fn2_5, fn3_5,Landmask,Aerosol_type,WV,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(6000))
    SR = np.full((6000,6000),np.nan)
    for i in range(6000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(6000,6000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'/'+date+'_b05.dat','wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band4 SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band5 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))
    
    
def SR_Band6():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','sir','02',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band5)(fn1_6, fn2_6, fn3_6,Landmask,Aerosol_type,WV,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(6000))
    SR = np.full((6000,6000),np.nan)
    for i in range(6000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(6000,6000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'/'+date+'_b06.dat','wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band4 SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band6 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))

In [8]:
for d in range(8,18):
    date_start = '2018-01-{} 00:00'.format(d)
    date_end = '2018-01-{} 09:00'.format(d)
    date_t = dt.timedelta(minutes=10)
    date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
    date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
    date_time_now = date_s
    print("Start {} ".format(date_start[:-6]))
    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
    CAMS_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS()
    CAMS_AERO_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS_AERO()
    while date_time_now <= date_e:
        
        date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
        YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)    
        print("start processing {}".format(date))
        # make dir
        folder_original = target + date +'_original/'
        folder_AC = target+date+'_AC'
        mkdir(folder_original)
        mkdir(folder_AC)
        
        S_Read_AHI_time = T.time()

        # Solar angle
        print('Start read solar angle')
        S_ANGLE_time = T.time()
        AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
        RAA = abs(AHI_SAA - AHI_VAA)
        RAA[RAA>180]=360-RAA[RAA>180]
        print('Finish read solar angle')
        E_ANGLE_time = T.time()

        # Atmosphereic data
        print('Start read CAMS')
        S_ATMOS_time = T.time()
        OZ,WV,AOT550 = CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).CAMS_Temporal(CAMS_ds)
        Aerosol_type = CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).CAMS_AERO_Temporal(CAMS_AERO_ds)
        print('Finish read CAMS')
        
        E_ATMOS_time = T.time()
        print('Read Atmospheric data cost : {}'.format(E_ATMOS_time-S_ATMOS_time))
        
        SR_Band1()
        SR_Band2()
        SR_Band3()
        SR_Band4()
       

        date_time_now = date_time_now + date_t

Start 2018-01-8 
start processing 201801080000
Start read solar angle
Finish read solar angle
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 19.687999963760376
Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR


<ipython-input-2-8be88e889747>:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Calculate SR finished
Start Save SR file 
Save Band1 SR finished
Band1 cost time: 57.1 secs, 1.0 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR


<ipython-input-2-8be88e889747>:51: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Calculate SR finished
Start Save SR file 
Save Band2 SR finished
Band2 cost time: 44.6 secs, 0.7 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR


...

<ipython-input-2-8be88e889747>:140: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Calculate SR finished
Start Save SR file 
Save Band4 SR finished
Band4 cost time: 40.9 secs, 0.7 mins,0.0 hours


start processing 201801110010
Start read solar angle
Finish read solar angle
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 20.773963689804077
Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band1 SR finished
Band1 cost time: 39.8 secs, 0.7 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band2 SR finished
Band2 cost time: 39.0 secs, 0.6 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band3 SR finished
Band3 cost time: 78.2 secs, 1.3 mins,0.0 hours


Start download AHI
Himawari8/AHI data Proces

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Finish read CAMS
Read Atmospheric data cost : 19.379281759262085
Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band1 SR finished
Band1 cost time: 38.6 secs, 0.6 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band2 SR finished
Band2 cost time: 39.3 secs, 0.7 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band3 SR finished
Band3 cost time: 75.2 secs, 1.3 mins,0.0 hours


Start download AHI
Himawari8/AHI data Processed Finish
data reading finish
Finish download AHI
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save Band4 SR finished
Band4 cost time: 39.1 secs, 0.7 mins,0.0 hours


start processing 201801

KeyboardInterrupt: 